In [ ]:
using DifferentialEquations
using Interact
using Plots
using Latexify

In [ ]:
function plot_sim(ode, u0, p)
    prob = ODEProblem(ode, u0, (0.,10.), p)
    sol = solve(prob, Rosenbrock23())
    plot(sol)
end

function iplot(ode)
    n_params = length(ode.params)
    n_vars = length(ode.syms)
    params = [vselection_slider(signif.(logspace(-3,3,201),3), label=latexify(p)) for p in ode.params]
    u0 = [vselection_slider(signif.(logspace(-3,3,201),3), label=latexify(p)) for p in ode.syms]
    display(hbox(params...))
    display(hbox(u0...))
    map((x...) -> plot_sim(ode, collect(x[1:n_vars]), collect(x[n_vars+1:end])),
        signal.(u0)..., signal.(params)...
    )
end

In [ ]:
ode = @reaction_network Demo1 begin
    (p, d), 0 ↔ x
end p d

In [ ]:
iplot(ode)